In [ ]:
def dqnb(n_episodes=epoch_no, max_t=1000, eps_start=1.0, eps_end=0.001, eps_decay=0.995,bench_score=1500000):
    """Deep Q-Learning.

    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of atimesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    epi_scores = []                        # list containing scores from each episode
    epi_task=[]
    epi_task_P1=[]
    epi_task_P2=[]
    epi_task_P3=[]


    epi_latency=[]
    epi_latency_P1=[]
    epi_latency_P2=[]
    epi_latency_P3=[]
    epi_task_select_P1=[]
    epi_task_select_P2=[]
    epi_task_select_P3=[]
    epi_datasize=[]
    epi_datasize_P1=[]
    epi_datasize_P2=[]
    epi_datasize_P3=[]

    epi_energy=[]
    epi_energy_P1=[]
    epi_energy_P2=[]
    epi_energy_P3=[]

    epi_num_task=[]
    epi_num_task_P1=[]
    epi_num_task_P2=[]
    epi_num_task_P3=[]

    mean_scores = [] # average score by the windos size
    mean_task=[] # average task ratio by window size
    mean_energy = [] # average energy cost by window size
    mean_latency=[]
    mean_datasize=[]

    mean_tasks_P1=[]
    mean_tasks_P2=[]
    mean_tasks_P3=[]

    mean_energy_P1=[]
    mean_energy_P2=[]
    mean_energy_P3=[]

    mean_latency_P1=[]
    mean_latency_P2=[]
    mean_latency_P3=[]


    mean_datasize_P1=[]
    mean_datasize_P2=[]
    mean_datasize_P3=[]
    reward_list_P1=[]
    reward_list_P2=[]
    reward_list_P3=[]

    scores_window = deque(maxlen=window_size)  # last score_window_size scores
    task_done_ratio_window = deque(maxlen=window_size)  # last  task dones
    energy_window = deque(maxlen=window_size)  # last  engery cost
    latency_window = deque(maxlen=window_size)  # last  engery cost
    datasize_window= deque(maxlen=window_size)


    mean_tasks_P1_window = deque(maxlen=window_size)  # last  engery cost
    mean_tasks_P2_window = deque(maxlen=window_size)  # last  engery cost
    mean_tasks_P3_window = deque(maxlen=window_size)  # last  engery cost

    win_num_task = deque(maxlen=window_size)
    win_num_task_P1 = deque(maxlen=window_size)  # last  engery cost
    win_num_task_P2 = deque(maxlen=window_size)  # last  engery cost
    win_num_task_P3 = deque(maxlen=window_size)  # last  engery cost

    mean_datasize_window=deque(maxlen=window_size)
    mean_datasize_P1_window= deque(maxlen=window_size)  # last  engery cost
    mean_datasize_P2_window = deque(maxlen=window_size)  # last  engery cost
    mean_datasize_P3_window = deque(maxlen=window_size)  # last  engery cost

    mean_energy_window_P1 = deque(maxlen=window_size)  # last  engery cost
    mean_energy_window_P2 = deque(maxlen=window_size)  # last  engery cost
    mean_energy_window_P3 = deque(maxlen=window_size)  # last  engery cost

    mean_latency_P1_window=deque(maxlen=window_size)
    mean_latency_P2_window=deque(maxlen=window_size)
    mean_latency_P3_window=deque(maxlen=window_size)

    bucket_means = {}
    bucket_task_len={}
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        mec_evn.reset() # reset the environment

        score = 0
        energy = []
        latency=[]
        task_done = []
        energy_P1=[]
        energy_P2=[]
        energy_P3=[]
        task_done_P1=[]
        task_done_P2=[]
        task_done_P3=[]
        latency_P1=[]
        latency_P2=[]
        latency_P3=[]
        datasize=[]
        datasize_P1=[]
        datasize_P2=[]
        datasize_P3=[]


        buckets = {range_tuple: {'P1_task_done': [], 'P2_task_done': [], 'P3_task_done': []} for range_tuple in bucket_ranges}

        tasks = mec_evn.get_task()
        for t in range(max_t):

            state = mec_evn.get_state(tasks)
            action = agent.act(state, eps)
            next_state, feedback,done,action_task,next_task = mec_evn.step(action,tasks)
            agent.step(state, action, feedback['reward'], next_state, done)

            score += feedback['reward']
            energy.append(feedback['energy'])
            datasize.append(action_task['data_size'])
            bucket = find_bucket(action_task['user_distance'], bucket_ranges)
            if feedback['task_done']==1:

              latency.append(feedback['latency'])


            if action_task['data_size'] != 0:
              task_done.append(feedback['task_done'])
              if action_task['priority']==1:
                  energy_P1.append(feedback['energy'])
                  task_done_P1.append(feedback['task_done'])
                  datasize_P1.append(action_task['data_size'])
                  reward_list_P1.append(feedback['reward'])
                  buckets[bucket]['P1_task_done'].append(feedback['task_done'])
                  if feedback['task_done']:
                    latency_P1.append(feedback['latency'])

              if action_task['priority']==2:
                  energy_P2.append(feedback['energy'])
                  task_done_P2.append(feedback['task_done']*feedback['task_done'])
                  datasize_P2.append(action_task['data_size'])
                  reward_list_P2.append(feedback['reward'])
                  buckets[bucket]['P2_task_done'].append(feedback['task_done'])
                  if feedback['task_done']:
                    latency_P2.append(feedback['latency'])

              if action_task['priority']==3:
                  energy_P3.append(feedback['energy'])
                  task_done_P3.append(feedback['task_done'])
                  datasize_P3.append(action_task['data_size'])
                  reward_list_P3.append(feedback['reward'])
                  buckets[bucket]['P3_task_done'].append(feedback['task_done'])
                  if feedback['task_done']:
                    latency_P3.append(feedback['latency'])
            

            
        
                # Append values to their respective lists within the bucket
                
               



            
            tasks=next_task
            if done:
                break
        

        for range_tuple, tasks in buckets.items():
            bucket_means[range_tuple] = {} 
            bucket_task_len[range_tuple]={}
            # Initialize a sub-dictionary for each range
            for task, values in tasks.items():
                # Calculate the mean for each task list
                mean_value = sum(values) / len(values) if values else 0  # Ensure division is valid
                mean_len=len(values)
                # Create a new key for storing the mean value of the task list
                mean_key = f'{task}_mean'
                len_key=f'{task}_mean_len'
                # Store the mean value in the corresponding range's sub-dictionary
                bucket_means[range_tuple][mean_key] = mean_value
                bucket_task_len[range_tuple][len_key]=mean_len

# Now, bucket_means contains only the mean values for each task list within each bucket range.
# Displaying the new bucket_means dictionary


        scores_window.append(score)       # save most recent score
        epi_scores.append(score)              # save most recent score
        mean_score = np.mean(scores_window)
        epi_task.append(np.mean(task_done))
        epi_task_P1.append(np.mean(task_done_P1))
        epi_task_P2.append(np.mean(task_done_P2))
        epi_task_P3.append(np.mean(task_done_P3))

        epi_task_select_P1.append(len(task_done_P1))
        epi_task_select_P2.append(len(task_done_P2))
        epi_task_select_P3.append(len(task_done_P3))

        epi_latency.append(np.mean(latency))
        epi_latency_P1.append(np.mean(latency_P1))
        epi_latency_P2.append(np.mean(latency_P2))
        epi_latency_P3.append(np.mean(latency_P3))

        epi_energy.append(np.mean(energy))
        epi_energy_P1.append(np.mean(energy_P1))
        epi_energy_P2.append(np.mean(energy_P2))
        epi_energy_P3.append(np.mean(energy_P3))

        epi_num_task.append(sum(task_done_P1)+sum(task_done_P2)+sum(task_done_P3))
        epi_num_task_P1.append(sum(task_done_P1))
        epi_num_task_P2.append(sum(task_done_P2))
        epi_num_task_P3.append(sum(task_done_P3))

        epi_datasize.append(np.mean(datasize))
        epi_datasize_P1.append(np.mean(datasize_P1))
        epi_datasize_P2.append(np.mean(datasize_P2))
        epi_datasize_P3.append(np.mean(datasize_P3))





        eps = max(eps_end, eps_decay*eps) # decrease epsilon

        print('\rEpisode {}\tAverage Score: {:.4f}\tmeantaskP1: {:.4f}\tmeantaskP2: {:.4f}\tmeantaskP3: {:.4f}\tmeanenergyP1: \
            {:.4f}\tmeanenergyP2: {:.4f}\tmeanenergyP3: {:.4f}\tmeanlatencyP1: {:.4f}\tmeanlatencyP2: {:.4f}\tmeanlatencyP3:{:.4f}'.format(i_episode, \
            mean_score, np.mean(task_done_P1), np.mean(task_done_P2), np.mean(task_done_P3), np.mean(energy_P1), np.mean(energy_P2), np.mean(energy_P3), \
                                    np.mean(latency_P1), np.mean(latency_P2), np.mean(latency_P3)), end="")





        if i_episode % window_size == 0:
            print(bucket_means)
            print(bucket_task_len)

            mean_scores.append(mean_score)
            task_done_ratio_window.append(np.mean(task_done))       # task done ratio
            mean_task.append(np.mean(task_done_ratio_window))
            energy_window.append(np.mean(energy))
            latency_window.append(np.mean(latency))

            mean_energy.append(np.mean(energy_window))
            mean_latency.append(np.mean(latency_window))


            mean_datasize_window.append(np.mean(datasize))
            mean_datasize.append(np.mean(mean_datasize_window))

            mean_tasks_P1_window.append(np.mean(task_done_P1))
            mean_tasks_P1.append(np.mean( mean_tasks_P1_window))

            mean_tasks_P2_window.append(np.mean(task_done_P2))
            mean_tasks_P2.append(np.mean( mean_tasks_P2_window))

            mean_tasks_P3_window.append(np.mean(task_done_P3))
            mean_tasks_P3.append(np.mean( mean_tasks_P3_window))

            mean_energy_window_P1.append(np.mean(energy_P1))
            mean_energy_P1.append(np.mean(mean_energy_window_P1))

            mean_energy_window_P2.append(np.mean(energy_P2))
            mean_energy_P2.append(np.mean(mean_energy_window_P2))

            mean_energy_window_P3.append(np.mean(energy_P3))
            mean_energy_P3.append(np.mean(mean_energy_window_P3))


            mean_latency_P1_window.append(np.mean(latency_P1))
            mean_latency_P1.append(np.mean( mean_latency_P1_window))

            mean_latency_P2_window.append(np.mean(latency_P2))
            mean_latency_P2.append(np.mean( mean_latency_P2_window))

            mean_latency_P3_window.append(np.mean(latency_P3))
            mean_latency_P3.append(np.mean( mean_latency_P3_window))

            mean_datasize_P1_window.append(np.mean(datasize_P1))
            mean_datasize_P1.append(np.mean(mean_datasize_P1_window))

            mean_datasize_P2_window.append(np.mean(datasize_P2))
            mean_datasize_P2.append(np.mean(mean_datasize_P2_window))

            mean_datasize_P3_window.append(np.mean(datasize_P3))
            mean_datasize_P3.append(np.mean(mean_datasize_P3_window))


            win_num_task.append(sum(task_done_P1)+sum(task_done_P2)+sum(task_done_P3))
            win_num_task_P1.append(sum(task_done_P1))
            win_num_task_P2.append(sum(task_done_P2))
            win_num_task_P3.append(sum(task_done_P3))


            print('\rEpisode {}\tAverage Score: {:.4f}\tmeantaskP1: {:.4f}\tmeantaskP2: {:.4f}\tmeantaskP3: {:.4f}\tmeanenergyP1: \
            {:.4f}\tmeanenergyP2: {:.4f}\tmeanenergyP3: {:.4f} \tmeanlatencyP1: {:.4f} \tmeanlatencyP2: {:.4f} \tmeanlatencyP3: {:.4f} \tmeandatasize: {:.4f} \tmeandatasizeP1: {:.4f} \tmeandatasizeP2: {:.4f} \tmeandatasizeP3: {:.4f}' .format(i_episode, mean_score,\
            mean_tasks_P1[-1],mean_tasks_P2[-1],mean_tasks_P3[-1],mean_energy_P1[-1],mean_energy_P2[-1],mean_energy_P3[-1], mean_latency_P1[-1],mean_latency_P2[-1],mean_latency_P3[-1],mean_datasize[-1],mean_datasize_P1[-1], \
                                                                                                                          mean_datasize_P2[-1],mean_datasize_P3[-1]



                                                                                                                                                                                                                                                       ))


            #print(len(task_done_P1),len(task_done_P2),len(task_done_P3))
           # print(len(task_done_P1)+len(task_done_P2)+len(task_done_P3))
            print('mean_task_till_this_window',np.mean(epi_task[-window_size:]), 'T1::',   np.mean(epi_task_P1[-window_size:]) ,'T2::' ,np.mean(epi_task_P2[-window_size:]) ,'T3::',np.mean(epi_task_P3[-window_size:]))
            #print('mean_energy_till_this_window',np.mean(mean_energy[-window_size:]), 'T1::', np.mean(mean_energy_P1[-window_size:])  ,'T2::', np.mean(mean_energy_P2[-window_size:])  ,'T3::',np.mean(mean_energy_P3[-window_size:]))
           # print('mean Latency till this window',np.mean(mean_latency[-window_size:]),'T1::', np.mean(mean_latency_P1[-window_size:]),'T1::', np.mean(mean_latency_P2[-window_size:]),'T1::', np.mean(mean_latency_P3[-window_size:]))
           # print(type(mean_tasks_P1),type(epi_num_task_P1))
            #print('mean_datasize', mean_datasize[-1])
            print( 'task_select_P1', np.mean(epi_task_select_P1[-window_size:]), 'task_select_P2', np.mean(epi_task_select_P2[-window_size:]),'task_select_P3', np.mean(epi_task_select_P3[-window_size:]))
            print('total_task_done_P1::', np.mean(epi_num_task_P1[-window_size:]),'total_task_done_P2::', np.mean(epi_num_task_P2[-window_size:]),'total_task_done_P3::',np.mean(epi_num_task_P3[-window_size:]))

            #print(np.mean(reward_list_P1), np.mean(reward_list_P2), np.mean(reward_list_P3))
            #print(min(reward_list_P1), min(reward_list_P2), min(reward_list_P3))
           # print(max(reward_list_P1), max(reward_list_P2), max(reward_list_P3))


            #print(mean_tasks_P1)
           # print(epi_task_P1)



        if np.mean(scores_window)>=bench_score or i_episode % n_episodes==0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.4f}'.format(i_episode, mean_score))
            torch.save(agent.qnetwork_local.state_dict(), model_path)
            break

    return epi_scores,epi_task,epi_energy,epi_latency,epi_num_task, epi_task_P1,epi_task_P2,epi_task_P3,epi_latency_P1,epi_latency_P2,epi_latency_P3,epi_energy_P1,epi_energy_P2,epi_energy_P3,epi_num_task_P1,epi_num_task_P2,epi_num_task_P3, \
    epi_task_select_P1,epi_task_select_P2,epi_task_select_P3,mean_scores,mean_task,mean_energy,mean_latency,win_num_task, mean_tasks_P1,mean_tasks_P2,mean_tasks_P3, mean_energy_P1,\
                 mean_energy_P2, mean_energy_P3,mean_latency_P1,mean_latency_P2,mean_latency_P3,win_num_task_P1,win_num_task_P2,win_num_task_P3,epi_datasize,epi_datasize_P1,epi_datasize_P2,epi_datasize_P3,\
                 mean_datasize,mean_datasize_P1,mean_datasize_P2,mean_datasize_P3

dqnb_epi_scores,dqnb_epi_task,dqnb_epi_energy,dqnb_epi_latency,dqnb_epi_num_task,dqnb_epi_task_P1,dqnb_epi_task_P2,dqnb_epi_task_P3,dqnb_epi_latency_P1,dqnb_epi_latency_P2,dqnb_epi_latency_P3,\
dqnb_epi_energy_P1,dqnb_epi_energy_P2,dqnb_epi_energy_P3,dqnb_epi_num_task_P1,dqnb_epi_num_task_P2,dqnb_epi_num_task_P3,dqnb_epi_task_select_P1,dqnb_epi_task_select_P2,dqnb_epi_task_select_P3,\
dqnb_mean_scores,dqnb_mean_task,dqnb_mean_energy,dqnb_mean_latency,dqnb_win_num_task, dqnb_mean_tasks_P1,dqnb_mean_tasks_P2,dqnb_mean_tasks_P3,\
dqnb_mean_energy_P1,dqnb_mean_energy_P2,dqnb_mean_energy_P3,dqnb_mean_latency_P1,dqnb_mean_latency_P2,dqnb_mean_latency_P3 ,dqnb_win_num_task_P1, dqnb_win_num_task_P2,dqnb_win_num_task_P3,\
dqnb_epi_datasize,dqnb_epi_datasize_P1,dqnb_epi_datasize_P2,dqnb_epi_datasize_P3,dqnb_mean_datasize,dqnb_mean_datasize_P1,dqnb_mean_datasize_P2,dqnb_mean_datasize_P3 = dqnb()